In [77]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt
import re


In [4]:
news_df = pd.read_csv('./FinalTrainSet.csv')
news_df

,Unnamed: 0,Headline,Body,Stance,Headline Word Count,Body Word Count
0,0,Police find mass graves with at least '15 bodi...,Danny Boyle is directing the untitled filmSeth...,unrelated,19,189
1,1,Hundreds of Palestinians flee floods in Gaza a...,Hundreds of Palestinians were evacuated from t...,agree,11,417
2,2,"Christian Bale passes on role of Steve Jobs, a...",30-year-old Moscow resident was hospitalized w...,unrelated,16,189
3,3,HBO and Apple in Talks for $15/Month Apple TV ...,(Reuters) - A Canadian soldier was shot at the...,unrelated,14,78
4,4,Spider burrowed through tourist stomach and up...,"Fear not arachnophobes, the story of Bunbury ""...",disagree,10,588
...,...,...,...,...,...,...
49967,49967,Urgent: The Leader of ISIL 'Abu Bakr al-Baghda...,(CNN) -- Boko Haram laughed off Nigeria announ...,unrelated,11,623
49968,49968,Brian Williams slams social media for speculat...,Along with unveiling the Apple Watch earlier t...,unrelated,14,216
49969,49969,Mexico Says Missing Students Not Found In Firs...,The bodies found in a mass grave were confirme...,agree,10,15
49970,49970,US Lawmaker: Ten ISIS Fighters Have Been Appre...,Rep. Duncan Hunter (R-CA) told Greta Van Suste...,discuss,13,106


In [7]:
news_df_test = pd.read_csv('./FinalTestSet.csv')
news_df_test

,Unnamed: 0,Headline,Body,Stance,Headline Word Count,Body Word Count
0,0,Ferguson riots: Pregnant woman loses eye after...,A RESPECTED senior French police officer inves...,unrelated,15,306
1,1,Crazy Conservatives Are Sure a Gitmo Detainee ...,Dave Morin social networking company Path is r...,unrelated,10,198
2,2,A Russian Guy Says His Justin Bieber Ringtone ...,A bereaved Afghan mother took revenge on the T...,unrelated,14,296
3,3,"Zombie Cat: Buried Kitty Believed Dead, Meows ...",Hewlett-Packard is officially splitting in two...,unrelated,10,527
4,4,Argentina President Adopts Boy to End Werewolf...,An airline passenger headed to Dallas was remo...,unrelated,8,103
...,...,...,...,...,...,...
25408,25408,The success of the Affordable Care Act is a hu...,"Congressional Republicans, evidently hoping th...",agree,15,1037
25409,25409,The success of the Affordable Care Act is a hu...,Did Obamacare work?It’s worth reflecting upon ...,discuss,15,1616
25410,25410,The success of the Affordable Care Act is a hu...,Millions may lose coverage next year if Congre...,disagree,15,1001
25411,25411,The success of the Affordable Care Act is a hu...,"Come November, the grim trudge across the incr...",disagree,15,892


In [12]:
print(news_df.Stance.unique())
print(news_df_test.Stance.unique())
news_df.shape

['unrelated' 'agree' 'disagree' 'discuss']
['unrelated' 'agree' 'discuss' 'disagree']


(49972, 6)

In [17]:
news_df = news_df[['Headline', 'Body', 'Stance', 'Headline Word Count','Body Word Count']]
news_df.shape

(49972, 5)

In [54]:
print(news_df.groupby('Stance').nunique())
print(news_df.groupby('Stance').count())

           Headline  Body  Stance  Headline Word Count  Body Word Count
Stance                                                                 
agree           933   828       1                   31              453
disagree        285   291       1                   25              222
discuss        1305  1279       1                   35              613
unrelated      1648  1051       1                   35              539
           Headline   Body  Headline Word Count  Body Word Count
Stance                                                          
agree          3678   3678                 3678             3678
disagree        840    840                  840              840
discuss        8909   8909                 8909             8909
unrelated     36545  36545                36545            36545


In [37]:
print(news_df.columns)

Index(['Headline', 'Body', 'Stance', 'Headline Word Count', 'Body Word Count'], dtype='object')


In [44]:
columns = ['Stance', 'Counts']
df_class = pd.DataFrame(columns=columns)

In [52]:
news_df.count(0)

Headline               49972
Body                   49972
Stance                 49972
Headline Word Count    49972
Body Word Count        49972
dtype: int64

In [53]:
news_df.count(1)

0        5
1        5
2        5
3        5
4        5
        ..
49967    5
49968    5
49969    5
49970    5
49971    5
Length: 49972, dtype: int64

In [72]:
df_text = news_df[['Headline','Body']]
df_text

,Headline,Body
0,Police find mass graves with at least '15 bodi...,Danny Boyle is directing the untitled filmSeth...
1,Hundreds of Palestinians flee floods in Gaza a...,Hundreds of Palestinians were evacuated from t...
2,"Christian Bale passes on role of Steve Jobs, a...",30-year-old Moscow resident was hospitalized w...
3,HBO and Apple in Talks for $15/Month Apple TV ...,(Reuters) - A Canadian soldier was shot at the...
4,Spider burrowed through tourist stomach and up...,"Fear not arachnophobes, the story of Bunbury ""..."
...,...,...
49967,Urgent: The Leader of ISIL 'Abu Bakr al-Baghda...,(CNN) -- Boko Haram laughed off Nigeria announ...
49968,Brian Williams slams social media for speculat...,Along with unveiling the Apple Watch earlier t...
49969,Mexico Says Missing Students Not Found In Firs...,The bodies found in a mass grave were confirme...
49970,US Lawmaker: Ten ISIS Fighters Have Been Appre...,Rep. Duncan Hunter (R-CA) told Greta Van Suste...


In [80]:
print(termNames, scores)

['body' 'headline'] [3906.17514421  770.17514421]


In [85]:
!pip install wordcloud
from wordcloud import STOPWORDS
STOPWORDS.add("rt")
STOPWORDS.add("s")
STOPWORDS.add("u")
STOPWORDS.add("amp")
STOPWORDS.add("th")
STOPWORDS.add("will")
STOPWORDS.add("t")
STOPWORDS.add("m")

  Using cached wordcloud-1.8.1-cp38-cp38-win_amd64.whl (155 kB)


In [86]:
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('sentiwordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mahima\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Mahima\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Mahima\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mahima\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mahima\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\Mahima\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiword

True

In [92]:
#!pip install re
#Cleaning text data 
import re
for i in range(len(df_text)):
    d = df_text.loc[i]['Headline']
    d=re.sub(r'http\S+', '', d) #remove links
    d=re.sub("[^a-zA-Z]", " ", d) #remove all characters except letters
    d=d.lower() #convert all words to lowercase
    d=nltk.word_tokenize(d) #split sentences into word
    d=[word for word in d if not word in STOPWORDS] #remove the stopwords
    lemma=nltk.WordNetLemmatizer() 
    d=[lemma.lemmatize(word) for word in d] #identify the correct form of the word in the dictionary
    d=" ".join(d)
    
    df_text.at[i,'Headline']=d

In [107]:
import re
for i in range(len(df_text)):
    d = df_text.loc[i]["Body"]
    d=re.sub(r'http\S+', '', d) #remove links
    d=re.sub("[^a-zA-Z]", " ", d) #remove all characters except letters
    d=d.lower() #convert all words to lowercase
    d=nltk.word_tokenize(d) #split sentences into word
    d=[word for word in d if not word in STOPWORDS] #remove the stopwords
    lemma=nltk.WordNetLemmatizer() 
    d=[lemma.lemmatize(word) for word in d] #identify the correct form of the word in the dictionary
    d=" ".join(d)
    
    df_text.at[i,"Body"]=d

In [130]:
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()

def token(text):
    txt = nltk.word_tokenize(text.lower())
    return [st.stem(word) for word in txt]

In [131]:
cv = CountVectorizer(lowercase=True, 
                     tokenizer=token, stop_words=STOPWORDS,
                     analyzer=u'word', min_df=4)

#tv = TfidfVectorizer(stop_words = STOPWORDS,
#                     lowercase=True, binary = True)

In [97]:
news_test = pd.read_csv('./FinalTrainSet.csv')
news_test_text = news_test[['Headline','Body']]
news_train_target = news_df[['Stance']]
news_test_target = news_test[['Stance']]

In [124]:

df_text.drop(columns= ['Text'])

,Headline,Body
0,police find mass graf least body near mexico t...,danny boyle directing untitled filmseth rogen ...
1,hundred palestinian flee flood gaza israel ope...,hundred palestinian evacuated home sunday morn...
2,christian bale pass role steve job actor repor...,year old moscow resident hospitalized wound in...
3,hbo apple talk month apple tv streaming servic...,reuters canadian soldier shot canadian war mem...
4,spider burrowed tourist stomach chest,fear arachnophobes story bunbury spiderman mig...
...,...,...
49967,urgent leader isil abu bakr al baghdadi killed...,cnn boko haram laughed nigeria announcement ce...
49968,brian williams slam social medium speculation ...,along unveiling apple watch earlier year compa...
49969,mexico say missing student found first mass graf,body found mass grave confirmed missing student
49970,u lawmaker ten isi fighter apprehended coming ...,rep duncan hunter ca told greta van susteren t...


In [134]:
def numeric_label(label_value):
    if((label_value == 'agree') or (label_value == 'discuss')) :
        return 0
    else:
        return 1
        

In [137]:
news_train_target['target']=news_train_target['Stance'].apply(numeric_label)
news_train_target
news_test_target['target']=news_test_target['Stance'].apply(numeric_label)
news_test_target

<ipython-input-137-22ad7985899f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_train_target['target']=news_train_target['Stance'].apply(numeric_label)
<ipython-input-137-22ad7985899f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_test_target['target']=news_test_target['Stance'].apply(numeric_label)


,Stance,target
0,unrelated,1
1,agree,0
2,unrelated,1
3,unrelated,1
4,disagree,1
...,...,...
49967,unrelated,1
49968,unrelated,1
49969,agree,0
49970,discuss,0


In [138]:
tv = TfidfVectorizer(stop_words = STOPWORDS,lowercase=True, binary = True)

In [141]:
vec_train = tv.fit_transform(df_text['Body'])
vec_test = tv.transform(news_test_text['Body'])


In [142]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import  accuracy_score, confusion_matrix, classification_report

rfc = RandomForestClassifier(n_estimators=100, n_jobs=-1)
rfc.fit(X = vec_train, y = news_train_target['target'])

y_pred_rfc=rfc.predict(vec_test)
#accuracy_score(y_pred_rfc, test_target)
rfc_report=classification_report(news_test_target['target'],y_pred_rfc)
print(rfc_report)

              precision    recall  f1-score   support

           0       0.75      0.51      0.61     12587
           1       0.85      0.94      0.90     37385

    accuracy                           0.83     49972
   macro avg       0.80      0.73      0.75     49972
weighted avg       0.83      0.83      0.82     49972



In [143]:
df_text

,Headline,Body,Text
0,police find mass graf least body near mexico t...,danny boyle directing untitled filmseth rogen ...,danny boyle directing untitled filmseth rogen ...
1,hundred palestinian flee flood gaza israel ope...,hundred palestinian evacuated home sunday morn...,hundred palestinian evacuated home sunday morn...
2,christian bale pass role steve job actor repor...,year old moscow resident hospitalized wound in...,year old moscow resident hospitalized wound in...
3,hbo apple talk month apple tv streaming servic...,reuters canadian soldier shot canadian war mem...,reuters canadian soldier shot canadian war mem...
4,spider burrowed tourist stomach chest,fear arachnophobes story bunbury spiderman mig...,fear arachnophobes story bunbury spiderman mig...
...,...,...,...
49967,urgent leader isil abu bakr al baghdadi killed...,cnn boko haram laughed nigeria announcement ce...,cnn boko haram laughed nigeria announcement ce...
49968,brian williams slam social medium speculation ...,along unveiling apple watch earlier year compa...,along unveiling apple watch earlier year compa...
49969,mexico say missing student found first mass graf,body found mass grave confirmed missing student,body found mass grave confirmed missing student
49970,u lawmaker ten isi fighter apprehended coming ...,rep duncan hunter ca told greta van susteren t...,rep duncan hunter ca told greta van susteren t...


In [147]:
bool_series_t = pd.notnull(df_text['Text'])
df_text[bool_series_t]

,Headline,Body,Text
0,police find mass graf least body near mexico t...,danny boyle directing untitled filmseth rogen ...,danny boyle directing untitled filmseth rogen ...
1,hundred palestinian flee flood gaza israel ope...,hundred palestinian evacuated home sunday morn...,hundred palestinian evacuated home sunday morn...
2,christian bale pass role steve job actor repor...,year old moscow resident hospitalized wound in...,year old moscow resident hospitalized wound in...
3,hbo apple talk month apple tv streaming servic...,reuters canadian soldier shot canadian war mem...,reuters canadian soldier shot canadian war mem...
4,spider burrowed tourist stomach chest,fear arachnophobes story bunbury spiderman mig...,fear arachnophobes story bunbury spiderman mig...
...,...,...,...
49967,urgent leader isil abu bakr al baghdadi killed...,cnn boko haram laughed nigeria announcement ce...,cnn boko haram laughed nigeria announcement ce...
49968,brian williams slam social medium speculation ...,along unveiling apple watch earlier year compa...,along unveiling apple watch earlier year compa...
49969,mexico say missing student found first mass graf,body found mass grave confirmed missing student,body found mass grave confirmed missing student
49970,u lawmaker ten isi fighter apprehended coming ...,rep duncan hunter ca told greta van susteren t...,rep duncan hunter ca told greta van susteren t...


In [156]:
vec_train1 = tv.fit_transform(news_df)
#vec_test1 = tv.transform(news_test_text['Body'])
print(vec_train)

  (0, 3838)	0.09302367102933824
  (0, 5810)	0.06883583264734407
  (0, 4392)	0.11127480323187187
  (0, 15686)	0.08656546019528552
  (0, 10587)	0.059796879800383486
  (0, 1279)	0.1716374114136562
  (0, 20868)	0.08650923997205659
  (0, 11795)	0.08389009211914651
  (0, 7059)	0.046525818670501734
  (0, 20531)	0.13798013969033046
  (0, 16036)	0.16530199899205159
  (0, 7348)	0.1312476248037876
  (0, 20909)	0.13416695792973307
  (0, 10384)	0.05916743993863568
  (0, 10938)	0.09230968495014431
  (0, 11701)	0.08357830802556737
  (0, 3170)	0.13416695792973307
  (0, 9942)	0.12324044068168724
  (0, 1239)	0.1159251130750763
  (0, 10652)	0.08229848095269617
  (0, 6901)	0.08799181012518204
  (0, 224)	0.13372436978333554
  (0, 11700)	0.08960807959773205
  (0, 812)	0.1171273315845864
  (0, 11083)	0.11562566576575585
  :	:
  (49971, 20564)	0.07281829844060121
  (49971, 8928)	0.08640888417692955
  (49971, 1840)	0.07197462893000973
  (49971, 11685)	0.08502570548901935
  (49971, 14161)	0.06568567298315195
  

In [157]:
vec_train1.shape
rfc = RandomForestClassifier(n_estimators=100, n_jobs=-1)
rfc.fit(X = vec_train
        , y = news_train_target['target'])

RandomForestClassifier(n_jobs=-1)

In [ ]:
rfc.